# Modelling Dynamical Systems 
## Diffusion-Limited Aggregation 

### Imports

In [102]:
import random
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib import colors
import matplotlib.animation as animation


### Functions generating saved GIFs and Images 

In [103]:
def DLAsimulation_images(folder_name, file_name, matrix_2D, DLAmap):
    
    if not os.path.isdir(f"{folder_name}"):
        os.mkdir(f"{folder_name}")
    
    fig, ax = plt.subplots()
    cax = ax.matshow(matrix_2D, interpolation='nearest', cmap=DLAmap)
    
    ax.set_title("DLA Cluster", fontsize=20)
    ax.set_xlabel("Direction, $x$", fontsize=15)
    ax.set_ylabel("Direction, $y$", fontsize=15)
    
    plt.savefig(f"{folder_name}/{file_name}.png", dpi=200)
    
    plt.close(fig)

-----
### 1. Single Seed Simulation 

In [104]:
def DLAsimulation_gif(folder_name, file_name, states, DLAmap):
    if not os.path.isdir(f"{folder_name}"):
        os.mkdir(f"{folder_name}")
    
    fig = plt.figure()
    frames = []
    for state in states:
        frames.append([plt.imshow(state, cmap=DLAmap)])
    video = animation.ArtistAnimation(fig, frames, interval=100)
    video.save(f"{folder_name}/{file_name}.gif")
    plt.close(fig)

In [105]:
def CreateRandomWalker(radius, seed_row, seed_col):

    theta = 2*np.pi*random.random()
    row=int(radius*np.cos(theta))+seed_row 
    col=int(radius*np.sin(theta))+seed_col 
    location=[row, col] 
    return location
    

In [106]:
def checkSurrounding(location, matrix_2D):

    FreeParticle = True 
    outCircle = False 
    OnEdge = False
    row = location[0]
    col = location[1]

    if (row == 0) or (col==len(matrix_2D)-1) or (row == len(matrix_2D)-1) or (col == 0):
        OnEdge = True
        
    if not OnEdge:

        if matrix_2D[row+1,col] == 1:
            FreeParticle = False
            if matrix_2D[row,col] == 2:
                outCircle = True
        elif matrix_2D[row-1,col] == 1:
            FreeParticle=False
            if matrix_2D[row,col]==2:
                outCircle=True
        elif matrix_2D[row,col+1]==1:
            FreeParticle=False
            if matrix_2D[row,col]==2:
                outCircle=True
        elif matrix_2D[row,col-1]==1:
            FreeParticle=False
            if matrix_2D[row,col]==2:
                outCircle=True

    return FreeParticle , outCircle , OnEdge
    

In [107]:
def move(location):
    
    probability = random.random()
    if probability<0.25:
        location = [location[0] - 1,location[1]]
    elif probability<0.5:
        location = [location[0] + 1,location[1]]
    elif probability<0.75:
        location = [location[0],location[1] + 1]
    else:
        location = [location[0],location[1] - 1]
        
    return location

In [108]:
def DLAsimulation(file_name, radius, needGif, save_step = 500):
    
    matrixLength = 2*radius + 5
    seed_row = radius + 2
    seed_col = radius + 2
    matrix_2D = np.zeros((matrixLength, matrixLength))
    
    col, row = np.meshgrid(np.arange(matrixLength), np.arange(matrixLength))

    matrix_2D[seed_row, seed_col] = 1

    distance_from_seed = np.sqrt((col - seed_col)**2 + (row - seed_row)**2)
    matrix_2D[distance_from_seed > radius] = 2
    
    randomWalkerCount = 0
    saveCounter = 0
    completeCluster = False
    maxWalkerCount = int(matrixLength**2)
    DLAmap = colors.ListedColormap(['black', 'yellow', 'white'])
    states = []
    
    while not completeCluster and randomWalkerCount < maxWalkerCount:
        
        location = CreateRandomWalker(radius, seed_row, seed_col)
        randomWalkerCount += 1
        saveCounter += 1
        
        if saveCounter > save_step:
            states.append(np.copy(matrix_2D))
            saveCounter = 0
        
        FreeParticle = True 
        OnEdge = False
    
        while FreeParticle and not OnEdge:
            FreeParticle, outCircle, OnEdge = checkSurrounding(location,matrix_2D)
            if OnEdge:
               randomWalkerCount -= 1
            if FreeParticle and not OnEdge:
                location = move(location)
            elif not FreeParticle:
                matrix_2D[location[0], location[1]] = 1
                if outCircle:
                    completeCluster = True
    
    if not needGif:            
        DLAsimulation_images("single_seed_graphics", f"{file_name}", matrix_2D, DLAmap)
         
    else:
        DLAsimulation_gif("single_seed_graphics", f"{file_name}", states, DLAmap)
        

**Single Seed Simulation 🌱**

We begin running the simulation for a **single seed positioned in the middle** of the figure. Our steps include:

- **GIF Creation**: Illustrates the process of *Diffusion-Limited Aggregation (DLA)* as particles move and attach to the aggregate.
- **Final State Image**: Captures the **end result** of the simulation, allowing for detailed observation and analysis of the pattern formation.

In [109]:
DLAsimulation("single_seed_gif", 100, True, 500)
DLAsimulation("single_seed_image", 100, False)


-----
### 2. Circle Seed Simulation 

In [125]:
def DLAsimulationCircleSeed(file_name, map_radius, seed_radius, needGif, save_step = 500):
    
    matrixLength = 2*map_radius + 5
    seed_row = map_radius + 2
    seed_col = map_radius + 2
    matrix_2D = np.zeros((matrixLength, matrixLength))
    
    col, row = np.meshgrid(np.arange(matrixLength), np.arange(matrixLength))

    distance_from_seed = np.sqrt((col - seed_col)**2 + (row - seed_row)**2)
    matrix_2D[distance_from_seed > map_radius] = 2
    matrix_2D[distance_from_seed < seed_radius] = 1
    DLAmap = colors.ListedColormap(['black', 'yellow', 'white'])

    randomWalkerCount = 0
    saveCounter = 0
    completeCluster = False
    maxWalkerCount = int(matrixLength**2)
    states = []
    
    while not completeCluster and randomWalkerCount < maxWalkerCount:
        
        location = CreateRandomWalker(map_radius, seed_row, seed_col)
        randomWalkerCount += 1
        saveCounter += 1
        
        if saveCounter > save_step:
            states.append(np.copy(matrix_2D))
            saveCounter = 0
        
        FreeParticle = True 
        OnEdge = False
    
        while FreeParticle and not OnEdge:
            FreeParticle, outCircle, OnEdge = checkSurrounding(location,matrix_2D)
            if OnEdge:
                randomWalkerCount -= 1
            if FreeParticle and not OnEdge:
                location = move(location)
            elif not FreeParticle:
                matrix_2D[location[0], location[1]] = 1
                if outCircle:
                    completeCluster = True
    
    if not needGif:            
        DLAsimulation_images("circle_seed_graphics", f"{file_name}",matrix_2D,DLAmap)
        
    else:
        DLAsimulation_gif("circle_seed_graphics", f"{file_name}", states, DLAmap)

**Circle Seed Simulation ⚪️🟢🔴**

We run the simulation for a **circle seed positioned in the middle** of the figure.

This time, we vary the **radius** of the circle seed. It takes the values *10, 30, and 50*.

As before, we create **GIFs** to illustrate the process as well as **Final State Images** to analyze the final pattern. 

In [126]:
DLAsimulationCircleSeed("radius_10_gif", 100, 10, True, 500)
DLAsimulationCircleSeed("radius_10_image", 100, 10, False)
DLAsimulationCircleSeed("radius_30_gif", 100, 30, True, 500)
DLAsimulationCircleSeed("radius_30_image", 100, 30, False)
DLAsimulationCircleSeed("radius_50_gif", 100, 50, True, 500)
DLAsimulationCircleSeed("radius_50_image", 100, 50, False)

-----
### 3. Coral Simulation 

In [112]:
def CreateRandomWalkerCorals(depth, matrix_length):

    if ((depth<0) or (depth > matrix_length-1)):
        raise IndexError("Depth is out of range.")
    else:
        col = random.randint(0, matrix_length-1)
    return [depth,col]
    

In [113]:
def checkSurroundingCorals(location, matrix_2D):

    FreeParticle = True 
    SurfaceLayer = False 
    OnEdge = False
    row = location[0]
    col = location[1]

    if (row == len(matrix_2D)-1) or (col==len(matrix_2D)-1) or (col == 0):
        OnEdge = True
        
    if not OnEdge:

        if matrix_2D[row+1,col] == 1:
            FreeParticle = False
            if matrix_2D[row,col] == 2:
                SurfaceLayer = True
        elif matrix_2D[row-1,col] == 1:
            FreeParticle=False
        elif matrix_2D[row,col+1]==1:
            FreeParticle=False
        elif matrix_2D[row,col-1]==1:
            FreeParticle=False

    return FreeParticle , SurfaceLayer , OnEdge
    

In [114]:
def moveCorals(location):
    
    probability = random.random()
    if probability<0.33:
        location = [location[0] + 1,location[1]]
    elif probability<0.67:
        location = [location[0],location[1] + 1]
    else:
        location = [location[0],location[1] - 1]
        
    return location

In [115]:
def DLAsimulationCorals(file_name, depth, matrixLength, number_of_seeds, needGif, save_step = 500):
    
    matrix_2D = np.zeros((matrixLength, matrixLength))
    
    col, row = np.meshgrid(np.arange(matrixLength), np.arange(matrixLength))

    if number_of_seeds < 0 or number_of_seeds > matrixLength-1:
        raise IndexError("Depth is out of range.")
    else:
        seeds = np.linspace(0,matrixLength-1,number_of_seeds,dtype=int)
       
    matrix_2D[matrixLength-1, seeds] = 1
    matrix_2D[row <= depth] = 2
    DLAmap = colors.ListedColormap(['#001f3f', '#FF7F50', '#FFDAB9'])
    
    randomWalkerCount = 0
    saveCounter = 0
    states = []
    completeCluster = False
    maxWalkerCount = int(matrixLength**2)
    
    while not completeCluster and randomWalkerCount < maxWalkerCount:
        
        location = CreateRandomWalkerCorals(depth,matrixLength)
        randomWalkerCount += 1
        saveCounter += 1
        
        if saveCounter > save_step:
            states.append(np.copy(matrix_2D))
            saveCounter = 0
        
        FreeParticle = True 
        OnEdge = False
    
        while FreeParticle and not OnEdge:
            FreeParticle , SurfaceLayer , OnEdge = checkSurroundingCorals(location,matrix_2D)
            if OnEdge:
               randomWalkerCount -= 1
            if FreeParticle and not OnEdge:
                location = moveCorals(location)
            elif not FreeParticle:
                matrix_2D[location[0], location[1]] = 1
                if SurfaceLayer:
                    completeCluster = True
    
    if not needGif:            
        DLAsimulation_images("coral_graphics", f"{file_name}",matrix_2D,DLAmap) 
        
    else:
        DLAsimulation_gif("coral_graphics", f"{file_name}", states, DLAmap)

**Coral Simulation 🪸**

We simulate **the formation of corals** from **multiple seeds** at the bottom of the ocean bed (bottom of the figure) as other points diffuse downwards from the ocean surface layer.

We vary the **depth** and the **number of seeds** on the floor.

The **depth** varies between *15, 30, 60, and 120*, whereas, the **number of seeds** varies between *10, 30, 70, and 150*.

As before, we create **GIFs** to illustrate the process as well as **Final State Images** to analyze the final pattern.

In [116]:
DLAsimulationCorals("depth30_seeds10_gif", 30,205,10,True,500)
DLAsimulationCorals("depth30_seeds10_image", 30,205,10,False)
DLAsimulationCorals("depth30_seeds70_gif", 30,205,70,True,500)
DLAsimulationCorals("depth30_seeds70_image", 30,205,70,False)
DLAsimulationCorals("depth30_seeds150_gif", 30,205,150,True,500)
DLAsimulationCorals("depth30_seeds150_image", 30,205,150,False)
DLAsimulationCorals("depth15_seeds30_gif", 15,205,30,True,500)
DLAsimulationCorals("depth15_seeds30_image", 15,205,30,False)
DLAsimulationCorals("depth60_seeds30_gif", 60,205,30,True,500)
DLAsimulationCorals("depth60_seeds30_image", 60,205,30,False)
DLAsimulationCorals("depth120_seeds30_gif", 120,205,30,True,500)
DLAsimulationCorals("depth120_seeds30_image", 120,205,30,False)

-----
### 4. Probabilistic Coral Simulation

In [117]:
def checkSurroundingProbabilisticCorals(location, matrix_2D, probability):

    FreeParticle = True 
    SurfaceLayer = False 
    OnEdge = False
    row = location[0]
    col = location[1]
    Direction = "NONE"

    if (row == len(matrix_2D)-1) or (col==len(matrix_2D)-1) or (col == 0):
        OnEdge = True
        
    if not OnEdge:

        if matrix_2D[row+1,col] == 1:
            if random.random() < probability:
                FreeParticle = False
                if matrix_2D[row,col] == 2:
                    SurfaceLayer = True
            else: 
                Direction = "UP"
        elif matrix_2D[row-1,col] == 1:
            if random.random() < probability:
                FreeParticle = False
            else: 
                Direction = "DOWN"
        elif matrix_2D[row,col+1] == 1:
            if random.random() < probability:
                FreeParticle = False
            else: 
                Direction = "LEFT"
        elif matrix_2D[row,col-1] == 1:
            if random.random() < probability:
                FreeParticle = False
            else: 
                Direction = "RIGHT"

    return FreeParticle, SurfaceLayer, OnEdge, Direction

In [118]:
def moveProbabilisticCorals(location, Direction):
    match Direction:
        case "NONE":
            probability = random.random()
            if probability<0.33:
                location = [location[0] + 1,location[1]]
            elif probability<0.67:
                location = [location[0],location[1] + 1]
            else:
                location = [location[0],location[1] - 1]
        case "UP":
            location = [location[0] - 1,location[1]]
        case "DOWN":
            location = [location[0] + 1,location[1]]
        case "RIGHT":
            location = [location[0],location[1] + 1]
        case "LEFT":
            location = [location[0],location[1] - 1]
       
    return location

In [127]:
def DLAsimulationProbabilisticCorals(file_name, depth, matrixLength, number_of_seeds, probability, needGif, save_step = 500):
    
    matrix_2D = np.zeros((matrixLength, matrixLength))
    
    col, row = np.meshgrid(np.arange(matrixLength), np.arange(matrixLength))

    if number_of_seeds < 0 or number_of_seeds > matrixLength-1:
        raise IndexError("Depth is out of range.")
    else:
        seeds = np.linspace(0,matrixLength-1,number_of_seeds,dtype=int)
       
    matrix_2D[matrixLength-1, seeds] = 1
    matrix_2D[row <= depth] = 2
    DLAmap = colors.ListedColormap(['#001f3f', '#FF7F50', '#FFDAB9'])            
    states = []
    saveCounter = 0
    
    randomWalkerCount = 0
    completeCluster = False
    maxWalkerCount = int(matrixLength**2)
    
    while not completeCluster and randomWalkerCount < maxWalkerCount:
        
        
        location = CreateRandomWalkerCorals(depth, matrixLength)

        randomWalkerCount += 1
        saveCounter +=1
        
        if saveCounter > save_step:
            states.append(np.copy(matrix_2D))
            saveCounter = 0
            
        
        FreeParticle = True 
        OnEdge = False
    
        while FreeParticle and not OnEdge:
            FreeParticle, SurfaceLayer, OnEdge, Direction = checkSurroundingProbabilisticCorals(location,matrix_2D,probability)
            if OnEdge:
                randomWalkerCount -= 1
            if FreeParticle and not OnEdge:
                location = moveProbabilisticCorals(location, Direction)
            elif not FreeParticle:
                matrix_2D[location[0], location[1]] = 1
                if SurfaceLayer:
                    completeCluster = True
    
    if not needGif:            
        DLAsimulation_images("coral_proba_graphics", f"{file_name}",matrix_2D,DLAmap)
    if needGif:
        DLAsimulation_gif("coral_proba_graphics", f"{file_name}", states, DLAmap)

**Probabilistic Coral Simulation 🐟🐠🐡**

We simulate **the formation of corals** from **multiple seeds** at the bottom of the ocean bed as before but this time each particle **aggregates with a given probablility p**. *The particles do not simply  to the existing structure.*

We vary only the **probability p**.

The **probability p** varies between *0.07, 0.25, and 0.6*.

As before, we create **GIFs** to illustrate the process as well as **Final State Images** to analyze the final pattern.

In [128]:
DLAsimulationProbabilisticCorals("depth30_seeds30_p07_gif",30,205,30,0.07,True,500)
DLAsimulationProbabilisticCorals("depth30_seeds30_p07_image",30,205,30,0.07,False)
DLAsimulationProbabilisticCorals("depth30_seeds30_p25_gif",30,205,30,0.25,True,500)
DLAsimulationProbabilisticCorals("depth30_seeds30_p25_image",30,205,30,0.25,False)
DLAsimulationProbabilisticCorals("depth30_seeds30_p60_gif",30,205,30,0.6,True,500)
DLAsimulationProbabilisticCorals("depth30_seeds30_p60_image",30,205,30,0.6,False)

-----
### 5. Probabilistic Single Seed Simulation 

In [121]:
def checkSurroundingProbabilistic(location, matrix_2D, probability):

    FreeParticle = True 
    outCircle = False 
    OnEdge = False
    row = location[0]
    col = location[1]
    Direction = "NONE"

    if (row == 0) or (col==len(matrix_2D)-1) or (row == len(matrix_2D)-1) or (col == 0):
        OnEdge = True
        
    if not OnEdge:

        if matrix_2D[row+1,col] == 1:
            if random.random() < probability:
                FreeParticle = False
                if matrix_2D[row,col] == 2:
                    outCircle = True
            else: 
                Direction = "UP"
        elif matrix_2D[row-1,col] == 1:
            if random.random() < probability:
                FreeParticle = False
                if matrix_2D[row,col] == 2:
                    outCircle = True
            else: 
                Direction = "DOWN"
        elif matrix_2D[row,col+1] == 1:
            if random.random() < probability:
                FreeParticle = False
                if matrix_2D[row,col] == 2:
                    outCircle = True
            else: 
                Direction = "LEFT"
        elif matrix_2D[row,col-1] == 1:
            if random.random() < probability:
                FreeParticle = False
                if matrix_2D[row,col] == 2:
                    outCircle = True
            else: 
                Direction = "RIGHT"

    return FreeParticle, outCircle, OnEdge, Direction

In [122]:
def moveProbabilistic(location, Direction):
    match Direction:
        case "NONE":
            probability = random.random()
            if probability<0.25:
                location = [location[0] - 1,location[1]]
            elif probability<0.5:
                location = [location[0] + 1,location[1]]
            elif probability<0.75:
                location = [location[0],location[1] + 1]
            else:
                location = [location[0],location[1] - 1]
        case "UP":
            location = [location[0] - 1,location[1]]
        case "DOWN":
            location = [location[0] + 1,location[1]]
        case "RIGHT":
            location = [location[0],location[1] + 1]
        case "LEFT":
            location = [location[0],location[1] - 1]
       
    return location

In [123]:
def DLAsimulationProbabilistic(file_name, radius, probability, needGif, save_step = 500):
    
    matrixLength = 2*radius + 5
    seed_row = radius + 2
    seed_col = radius + 2
    matrix_2D = np.zeros((matrixLength, matrixLength))
    
    col, row = np.meshgrid(np.arange(matrixLength), np.arange(matrixLength))

    matrix_2D[seed_row, seed_col] = 1

    distance_from_seed = np.sqrt((col - seed_col)**2 + (row - seed_row)**2)
    matrix_2D[distance_from_seed > radius] = 2
    DLAmap = colors.ListedColormap(['black', 'yellow', 'white'])            
    
    randomWalkerCount = 0
    completeCluster = False
    maxWalkerCount = int(matrixLength**2)
    states = []
    saveCounter = 0 
    
    while not completeCluster and randomWalkerCount < maxWalkerCount:
        
        randomWalkerCount += 1
        saveCounter += 1
        location = CreateRandomWalker(radius, seed_row, seed_col)
        
        FreeParticle = True 
        OnEdge = False
        
        if saveCounter > save_step:
            states.append(np.copy(matrix_2D))
            saveCounter = 0
    
        while FreeParticle and not OnEdge:
            FreeParticle, outCircle, OnEdge, Direction = checkSurroundingProbabilistic(location,matrix_2D,probability)
            if OnEdge:
                randomWalkerCount -= 1
            if FreeParticle and not OnEdge:
                location = moveProbabilistic(location, Direction)
            elif not FreeParticle:
                matrix_2D[location[0], location[1]] = 1
                if outCircle:
                    completeCluster = True
        
    if not needGif:            
        DLAsimulation_images("single_seed_proba_graphics", f"{file_name}",matrix_2D,DLAmap)
    if needGif:
        DLAsimulation_gif("single_seed_proba_graphics", f"{file_name}", states, DLAmap)

**Probabilistic Single Seed Simulation 🪴**

We run the first simulation with a **centrally positionned single** seed but this time each particle **aggregates with a given probablility p**. *The particles do not simply  to the existing structure.*

We vary only the **probability p**.

The **probability p** varies between *0.07, 0.25, and 0.6*.

As before, we create **GIFs** to illustrate the process as well as **Final State Images** to analyze the final pattern.


In [124]:
DLAsimulationProbabilistic("single_seed_p07_gif", 100, 0.07, True, 500)
DLAsimulationProbabilistic("single_seed_p07_image", 100, 0.07, False)
DLAsimulationProbabilistic("single_seed_p25_gif", 100, 0.25, True, 500)
DLAsimulationProbabilistic("single_seed_p25_image", 100, 0.25, False)
DLAsimulationProbabilistic("single_seed_p60_gif", 100, 0.6, True, 500)
DLAsimulationProbabilistic("single_seed_p60_image", 100, 0.6, False)

-----
-----